In [55]:
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
import requests
import io
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
imputer_num = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
imputer_cat = SimpleImputer(strategy="most_frequent")
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from flytekit import task, workflow
from sklearn.preprocessing import LabelEncoder
hi=None


In [56]:
url = "https://github.com/smadarab/flytelab/raw/main/census.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')),sep=',')
print("df is created",df.columns)
#df.dropna(inplace=True)
#df = df.reset_index()
print(df.columns)
train = df

df is created Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')
Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')


In [57]:
#df = pd.read_csv("census 2.csv")

In [58]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from joblib import dump, load


In [18]:
ohe = OneHotEncoder(handle_unknown = 'ignore')
num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loos', 'hour-per-week']
cat_cols = ['workclass', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']
log_transform_cols = ['capital-loos', 'capital-gain']    
def get_cat_cols(X):
    return X[cat_cols]
def get_num_cols(X):
    return X[num_cols]
def get_log_transform_cols(X):
    return X[log_transform_cols]
def get_dummies(X):
    print('\n \n',type(X))
    return pd.get_dummies(pd.DataFrame(X))
def one_hot_encode(X):
    print("one_hot_encode")
    global hi
    ohe = OneHotEncoder(handle_unknown = 'ignore')
    print(X[0].shape)
    ohe.fit(pd.DataFrame(X))
    hi=ohe
    dump(ohe, 'onehot.joblib') 
    import subprocess
    subprocess.call(["git", "add","."])
    subprocess.call(["git", "commit","-m","kfbjebfe"])
    subprocess.call(["git", "push"])
    y = ohe.transform(pd.DataFrame(X)).toarray()
    return (y)
def label_encode(X):
    print("label_encode")
    df = pd.DataFrame(X)
    le = LabelEncoder()
    df[cols] = df[cols].apply(LabelEncoder().fit_transform)
    df.status = le.fit_transform(X)
def cat_imputer(X):
    print(X.shape)
    return(imputer_cat.fit_transform(X))
    #return X.apply(lambda col: imputer_cat.fit_transform(col))  

log_transform_pipeline = Pipeline([
('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),   
('log_transform', FunctionTransformer(np.log1p))
])

num_cols_pipeline = Pipeline([
('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),
('min_max_scaler', MinMaxScaler())
])

cat_cols_pipeline = Pipeline([
('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
('imputer', SimpleImputer(strategy="most_frequent")),
#('get_dummies', FunctionTransformer(get_dummies, validate=False))
('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))    
])       

steps_ = FeatureUnion([
('log_transform', log_transform_pipeline),
('num_cols', num_cols_pipeline),
('cat_cols', cat_cols_pipeline)
])


In [19]:
full_pipeline = Pipeline([('steps_', steps_)])
y = train['income'].map({' <=50K': 0, ' >50K': 1})
X= full_pipeline.fit_transform(train)

one_hot_encode
(7,)
[my_project 9f6fe2b] kfbjebfe
 2 files changed, 45232 insertions(+), 8 deletions(-)
 create mode 100755 projects/my_project/my_project/census 2.csv


To https://github.com/smadarab/flytelab
   e04445e..9f6fe2b  my_project -> my_project


In [20]:
model = AdaBoostClassifier(n_estimators=300)
X_train, X_test, y_train, y_test = train_test_split(X, y)
model.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=300)

In [22]:
y_pred = model.predict(X_test)

In [23]:
y_pred

array([0, 1, 1, ..., 0, 0, 0])

In [24]:
#print("score train: {}".format(roc_auc_score(y_train, probs_train)))
print("score test: {}".format(roc_auc_score(y_test, y_pred)))

score test: 0.7890661718639754


In [25]:

#Self-emp-not-inc HS-grad 9.0 Married-civ-spouse Exec-managerial Husband White Male 0.0 0.0 45.0 United-States

In [45]:
age =  1
education_num =  1.0
capital_gain =  0.0
capital_loos = 0.0
hour_per_week = 1.0
workclass = "Self-emp-not-inc"
marital_status = "Married-civ-spouse"
occupation = "Exec-managerial"
relationship = "Husband"
race = "White"
sex = "Male"
native_country ="United-States"
final_weight = 77516
education="Bachelors"

In [46]:
dict_val = {'age':age, 'workclass':workclass, 'final-weight':final_weight,  'education_num':education_num,
       'martial_status':marital_status, 'occupation':occupation, 'relationship':relationship, 'race':race, 'sex':sex,
       'capital-gain':capital_gain, 'capital_loos':capital_loos, 'hour_per_week':hour_per_week, 'native_country':native_country}
       

In [47]:
X_train = pd.DataFrame(dict_val,index=[0])

In [48]:
X_train

,age,workclass,final-weight,education_num,martial_status,occupation,relationship,race,sex,capital-gain,capital_loos,hour_per_week,native_country
0,1,Self-emp-not-inc,77516,1.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,1.0,United-States


In [49]:
import os

In [50]:
num_cols = ['age', 'education_num', 'capital-gain',
            'capital_loos', 'hour_per_week']
cat_cols = ['workclass', 
            'martial_status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native_country']
log_transform_cols = ['capital_loos', 'capital-gain']    
def get_cat_cols(X):
    return X[cat_cols]
def get_num_cols(X):
    return X[num_cols]
def get_log_transform_cols(X):
    print("in function",X.columns)
    return X[log_transform_cols]
def get_dummies(X):
    print('\n \n',type(X))
    return pd.get_dummies(pd.DataFrame(X))
def cat_imputer(X):
    print(X.shape)
    return(imputer_cat.fit_transform(X))
    #return X.apply(lambda col: imputer_cat.fit_transform(col))  
def one_hot_encode(X):
    print(X.shape)
    print("current wd",os.getcwd())
    #url = "https://github.com/smadarab/flytelab/blob/my_project/projects/my_project/my_project/onehot.joblib" # Make sure the url is the raw version of the file on GitHub
    #download = requests.get(url).content
    #ohe = load(download)
    #ohe = load('onehot.joblib')
    return hi.transform(pd.DataFrame(X)).toarray()

log_transform_pipeline = Pipeline([
('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),   
('log_transform', FunctionTransformer(np.log1p))
])

num_cols_pipeline = Pipeline([
('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),
('min_max_scaler', MinMaxScaler())
])

cat_cols_pipeline = Pipeline([
('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
('imputer', SimpleImputer(strategy="most_frequent")),
#('get_dummies', FunctionTransformer(get_dummies, validate=False))
('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))
])       

steps_ = FeatureUnion([
('log_transform', log_transform_pipeline),
('num_cols', num_cols_pipeline),
('cat_cols', cat_cols_pipeline)
])
full_pipeline = Pipeline([('steps_', steps_)])
X_train = full_pipeline.fit_transform(X_train)

in function Index(['age', 'workclass', 'final-weight', 'education_num', 'martial_status',
       'occupation', 'relationship', 'race', 'sex', 'capital-gain',
       'capital_loos', 'hour_per_week', 'native_country'],
      dtype='object')
(1, 7)
current wd /Users/apple/flytelab/projects/my_project/my_project


In [51]:
X_train.shape

(1, 93)

In [52]:
y_pred = model.predict_proba(X_train)

In [54]:
y_pred

array([[0.56617201, 0.43382799]])

In [53]:
if (y_pred.max())> 0.5:
    print(1)
elif     (y_pred.max())< 0.5:
    print(0)

1


In [ ]:
 ohe = load(download)

In [ ]:
ohe.transform(pd.DataFrame(X)).toarray()

# Test part

In [ ]:
num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loos', 'hour-per-week']
cat_cols = ['workclass', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']
log_transform_cols = ['capital-loos', 'capital-gain']    
def get_cat_cols(X):
    return X[cat_cols]
def get_num_cols(X):
    return X[num_cols]
def get_log_transform_cols(X):
    return X[log_transform_cols]
def get_dummies(X):
    print('\n \n',type(X))
    return pd.get_dummies(pd.DataFrame(X))
def one_hot_encode(X):
    print("one_hot_encode")
    ohe = load('filename.joblib') 
    y = ohe.transform(pd.DataFrame(X)).toarray()
    print('\n \n',y)
    return y
def label_encode(X):
    print("label_encode")
    df = pd.DataFrame(X)
    le = LabelEncoder()
    df[cols] = df[cols].apply(LabelEncoder().fit_transform)
    df.status = le.fit_transform(X)
def cat_imputer(X):
    print(X.shape)
    return(imputer_cat.fit_transform(X))
    #return X.apply(lambda col: imputer_cat.fit_transform(col))  

log_transform_pipeline = Pipeline([
('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),   
('log_transform', FunctionTransformer(np.log1p))
])

num_cols_pipeline = Pipeline([
('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),
('min_max_scaler', MinMaxScaler())
])

cat_cols_pipeline = Pipeline([
('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
('imputer', SimpleImputer(strategy="most_frequent")),
#('get_dummies', FunctionTransformer(get_dummies, validate=False))
('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))    
])       

steps_ = FeatureUnion([
('log_transform', log_transform_pipeline),
('num_cols', num_cols_pipeline),
('cat_cols', cat_cols_pipeline)
])

In [ ]:
age =  50
education_num =  13.0
capital_gain =  0.0
capital_loos = 0.0
hour_per_week = 13.0
workclass = "Self-emp-not-inc"
marital_status = "Married-civ-spouse"
occupation = "Exec-managerial"
relationship = "Husband"
race = "White"
sex = "Male"
native_country ="United-States"
final_weight = 77516
education="Bachelors"

In [ ]:
dict_val = {'age':age, 'workclass':workclass, 'final-weight':final_weight, 'education':education, 'education-num':education_num,
       'marital-status':marital_status, 'occupation':occupation, 'relationship':relationship, 'race':race, 'sex':sex,
       'capital-gain':capital_gain, 'capital-loos':capital_loos, 'hour-per-week':hour_per_week, 'native-country':native_country}
       

In [ ]:
X_train = pd.DataFrame(dict_val,index=[0])

In [ ]:
X_train

In [ ]:
full_pipeline = Pipeline([('steps_', steps_)])
X = full_pipeline.fit_transform(X_train)

In [ ]:
y = model.predict(X)

In [ ]:
y